In [23]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [24]:
from scripts.IG_Scraper import InstagramBot
from scripts.dataframe_functions import *

# Overview of scraping with the InstagramBot()

InstagramBot scrapes Instagram posts and their associated metrics from lists of users, then organizes then in a .csv file. Checks have been introduced to circumvent access blocking by taking actions at speeds more similar to the average, while also minimizing abnormal behaviour (e.g. repeatedly accessing the same pages, refreshing pages unecessarily etc.).  To that end, the bot will prioritize skipping a post when issues arise, but these can be filled in later during the cleanup stage.

Choosing a new niche to scrape involves the following steps:

1) Import or create a list of accounts that are representative of your niche

2) Create an InstagramBot() instance  (e.g.  IB)

3) Sign into your scraping account ->  IB.signIn()

4) Gather the posts of those users  -> IB.posts_from_list_of_users()

5) Fill in the dataframe ->  IB.fill_in_dataframe()   Note: the previous step gathers user, so pass include_user = False.  
                Additional Note: you may wish to run this twice to catch any posts that were missed during the first round.
                Additional Note: In the event of issues arising during the scraping process (e.g. you must shut down the computer), a temporary csv file will be exported every 20 posts and can be reloaded to resume scraping at a later time.
                
6) Post-processing of the dataframe  -> post_processing(df)

## Step 0: Choose the data folder for your scraping

In [25]:
"""
Main Variables:

root_directory: where all of the files related to this particular niche will be stored.
image_directory: where all of the images will be saved
dataframe_directory: where imports/outputs will be located
dataframe_name: desired dataframe name


"""

#root_dir = Path(r'C:\Users\William\Google Drive (william.dl.cameron@gmail.com)\Insight Project')

root_dir = Path(r'C:\Users\hliu39\Desktop\ins_image')

image_directory = root_dir/'Images'; image_directory.mkdir(exist_ok = True, parents = True)
df_dir = root_dir/'Output_Guys'; image_directory.mkdir(exist_ok = True, parents = True)


df_name = 'food_dataframe.csv'


df_path = df_dir/df_name
df = load_dataframe(df_path) if df_path.exists() else pd.DataFrame()

## Step 1: Fill in your account details

In [27]:
username = 'webscraper100'
password = 'ScrapingTheWeb100'

driver = webdriver.Chrome(r"C:/Users/hliu39/Downloads/chromedriver_win32/chromedriver.exe")

IB = InstagramBot(username,password)
IB.signIn()

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
""" Run this if you need to refresh your InstagramBot instance in the same browser (ex. code change)"""
IB = InstagramBot(username,password, IB.browser)

## Step 2: Gather posts and related information

In [14]:
#create a userlist from a text file
with open(Path('userlists')/'FoodUsers.txt') as f:
    user_list = [line.rstrip('\n') for line in f]

    
"""Alternatively, you can manually create your own.  Use a set to avoid duplicates"""
# user_list = {'localhaven',
#                 'feedtheswimmers',
#                 'smittenkitchen',
#                 'wellandfull',
#                }
# user_list = list(set(user_list))


'Alternatively, you can manually create your own.  Use a set to avoid duplicates'

In [15]:
df = IB.posts_from_list_of_users(user_list, df = df)
export_df(df, df_dir/f'prefilled_{df_name}')

NameError: name 'IB' is not defined

In [ ]:
df = load_dataframe(df_dir/f'prefilled_{df_name}')

In [ ]:
"""Step 3: Scrape information from the posts"""

df = IB.fill_in_dataframe(df, include_user = False, output_folder = df_dir)

In [ ]:
df.to_csv(df_dir/'filled_dataframe.csv')

In [ ]:
df

## Step 3: Process the Dataframe (Data Engineering)

In [67]:
df = load_dataframe(df_dir/'filled_dataframe.csv')

In [ ]:
df = post_processing(df, df_dir)

In [77]:
df.to_csv(df_dir/'processed_dataframe.csv')

In [78]:
df.columns

Index(['Links', 'filename', 'followers', 'following', 'num_posts', 'username',
       'likes', 'posttime', 'alt-text', 'caption', 'tags',
       'Engagement (Avg Likes)', 'Downloaded', 'engagement_factor_avg_likes',
       'rolling_avg', 'engagement_factor_moving_avg', 'date', 'hour', 'minute',
       'second', 'month', 'year', 'day_name', 'performance', 'red', 'green',
       'blue', 'brightness', 'red_range', 'green_range', 'blue_range',
       'contrast', 'rolling', 'engagement_factor_std',
       'rel_contrast_moving_avg', 'rel_brightness_moving_avg'],
      dtype='object')

# Playground - For Testing out new Functions